In [ ]:
import csv
import cv2
import numpy as np
import h5py
import json
import numpy as np

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers import MaxPooling2D
from keras.optimizers import Adam

relu = 'relu'

lines = []
with open('own_training_data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(lines, test_size=0.2)
      
images = []
measurements = []
for line in lines:
    center_path = line[0]
    left_path = line[1]
    right_path = line[2]
#Adding center images
    filename_c = center_path.split('/')[-1]
    current_path_c = './own_training_data/IMG/' + filename_c
    image_c = cv2.imread(current_path_c)
    images.append(image_c)
#Adding left images
    filename_l = left_path.split('/')[-1]
    current_path_l = './own_training_data/IMG/' + filename_l
    image_l = cv2.imread(current_path_l)
    images.append(image_l)
#Adding right images
    filename_r = right_path.split('/')[-1]
    current_path_r = './own_training_data/IMG/' + filename_r
    image_r = cv2.imread(current_path_r)
    images.append(image_r)
#Adding steering angles from all of the three views
    correction=0.2
    steer = line[3]
    steer_left=float(steer)+float(correction)
    steer_right=float(steer)-float(correction)
    measurements.append(steer)
    measurements.append(steer_left)
    measurements.append(steer_right)

augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(float(measurement)*-1.0)

X_train = np.array(augmented_images)
Y_train = np.array(augmented_measurements)

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,15),(1,1))))
# starts with five convolutional and maxpooling layers
model.add(Convolution2D(24, 5, 5, border_mode='same', subsample=(2, 2)))
model.add(Activation(relu))
model.add(MaxPooling2D(pool_size=(1, 1), strides=(1, 1)))
model.add(Convolution2D(36, 5, 5, border_mode='same', subsample=(2, 2)))
model.add(Activation(relu))
model.add(MaxPooling2D(pool_size=(1, 1), strides=(1, 1)))
model.add(Convolution2D(48, 5, 5, border_mode='same', subsample=(2, 2)))
model.add(Activation(relu))
model.add(MaxPooling2D(pool_size=(1, 1), strides=(1, 1)))
model.add(Convolution2D(64, 3, 3, border_mode='same', subsample=(1, 1)))
model.add(Activation(relu))
model.add(MaxPooling2D(pool_size=(1, 1), strides=(1, 1)))
model.add(Convolution2D(64, 3, 3, border_mode='same', subsample=(1, 1)))
model.add(Activation(relu))
model.add(MaxPooling2D(pool_size=(1, 1), strides=(1, 1)))
model.add(Flatten())
model.add(Dense(100))
model.add(Activation(relu))
model.add(Dense(50))
model.add(Activation(relu))
model.add(Dense(10))
model.add(Activation(relu))
model.add(Dense(1))
model.summary()
model.compile(optimizer=Adam(0.0001), loss="mse")

model.fit(X_train, Y_train, validation_split=0.2, shuffle=True, nb_epoch=6)

model.save('model.h5', True)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 160, 320, 3)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
cropping2d_2 (Cropping2D)        (None, 95, 318, 3)    0           lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_6 (Convolution2D)  (None, 48, 159, 24)   1824        cropping2d_2[0][0]               
____________________________________________________________________________________________________
activation_9 (Activation)        (None, 48, 159, 24)   0           convolution2d_6[0][0]            
___________________________________________________________________________________________